# ADNI and the K-operator

Using the visualization of connectivity matrices mutuated from Sidney Hough, Julian Quevedo, and Pino Cholsaipant (https://i.imgur.com/FYaEgKW.jpg).

Here: application to the ADNI dataset (Alzheimer-Perusini) disease.

Here: MSDL atlas

All the imports and installs we'll need.


In [ ]:
import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d
from torch_geometric.nn import EdgeConv, GCNConv, GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_matrix

In [ ]:
import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d
from torch_geometric.nn import EdgeConv, GCNConv, GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_matrix

In [ ]:

from nilearn import datasets, input_data
import os
import numpy as np
import nibabel as nib
from nilearn import datasets, input_data, connectome
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np


atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps
atlas_labels = atlas.labels


import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom


# Other patient: 002_S_5018, ADNI, M, 73.4, 73.7, 73.9, 74.5, Resting State fMRI
dicom_folder = "/Users/mariamannone/Downloads/ADNI 4/002_S_5018/Resting_State_fMRI/2012-11-08_07_20_35.0/I346237"

output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_5/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)

# Adapting something using my codes 

In [ ]:
from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker


nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_5/output.nii.gz/"


# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series.append(ts)

        # Extract the label from the file name or metadata
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series):
    np.savetxt(f'time_series_{i:04d}.csv', ts, delimiter=',')

Calculating partial correlation matrix for each time series and taking the mean. This will serve as the basis for edges in the graphs we create later.



In [ ]:
corr_measure = ConnectivityMeasure(kind='correlation')
pcorr_measure = ConnectivityMeasure(kind='partial correlation')

corr_matrices = corr_measure.fit_transform(time_series)
pcorr_matrices = pcorr_measure.fit_transform(time_series)


# STORING the information for learning & prediction

male_BL = corr_matrices

Let's visualize a functional connectome!

In [ ]:
# Partial correlation matrix
plotting.plot_matrix(corr_matrices[0], labels=atlas_labels, colorbar=True, vmax=1, vmin=-1)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices[0], coords, edge_threshold="80%", colorbar=True)

In [ ]:
# April 19, clear command

del time_series
del corr_matrices
del pcorr_matrices
del connectivity_matrix, correlation_matrix


In [ ]:
# Following time point


import os
import numpy as np
import nibabel as nib
from dcmstack import NiftiWrapper

dicom_folder = "/Users/mariamannone/Downloads/ADNI 4/002_S_5018/Resting_State_fMRI/2013-02-11_07_46_12.0/I358614"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_6"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)

In [ ]:

from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker


nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_6/output.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series):
    np.savetxt(f'time_series_{i:04d}.csv', ts, delimiter=',')

In [ ]:
corr_measure_fu1 = ConnectivityMeasure(kind='correlation')
pcorr_measure_fu1 = ConnectivityMeasure(kind='partial correlation')

corr_matrices_fu1 = corr_measure_fu1.fit_transform(time_series)
pcorr_matrices_fu1 = pcorr_measure_fu1.fit_transform(time_series)

# STORING the information for learning & prediction

male_FU = corr_matrices_fu1 


In [ ]:
# Partial correlation matrix
plotting.plot_matrix(corr_matrices_fu1[0], labels=atlas_labels, colorbar=True, vmax=1, vmin=-1)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices_fu1[0], coords, edge_threshold="80%", colorbar=True)

In [ ]:
# April 19, clear command

del time_series
del corr_matrices_fu1
del pcorr_matrices_fu1
del connectivity_matrix_fu1, correlation_matrix_fu1, 


In [ ]:
###


# K between these two time points


###




import numpy as np


matrix1 = male_BL[0] # corr_matrices[0]

# Load matrix computed above for the AP patient
matrix2 = male_FU[0] #corr_matrices_fu1[0]

# Compute the inverse of the matrix
inverse_matrix1 = np.linalg.inv(matrix1)
print(inverse_matrix1)


K_approximation = np.dot(matrix2,inverse_matrix1)

# print(K_approximation)

# Plots

vmin = -1
vmax = 1


# K-operator
plotting.plot_matrix(K_approximation, labels = atlas_labels, colorbar=True, vmax=1, vmin=-1) # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_approximation, coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)



# Check if matrix1_approx is close to matrix2
tolerance = 1e-6
is_close = np.allclose(matrix2, np.dot(K_approximation,matrix1), atol=tolerance)

if is_close:
    print("KG is close to G^p.")
else:
    print("KG is not close to G^p.")
    

In [ ]:
# April 10 -- re-ordering the ROIs


import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting


# Define a dictionary to map each ROI to its lobe
lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Reorder ROI labels according to lobes
reordered_labels = []
for lobe in lobe_mapping:
    reordered_labels.extend(lobe_mapping[lobe])

# Now, plot using the reordered labels
plotting.plot_matrix(K_approximation, labels=reordered_labels, colorbar=True, vmax=1, vmin=-1)  # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_approximation, coords, edge_threshold="80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)


# Reorder ROI labels according to lobes
reordered_labels = []
for lobe in lobe_mapping:
    reordered_labels.extend(lobe_mapping[lobe])

# Now, plot using the reordered labels
plotting.plot_matrix(K_approximation, labels=reordered_labels, colorbar=True, vmax=1, vmin=-1)  # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_approximation, coords, edge_threshold="80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)


In [ ]:
#####


# Block-diagonal representation of K


#####



# Addendum of March 26

import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

# Define a custom mapping to group ROIs by brain lobes
lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}


# Load matrices computed for the control and AP patients
matrix1 = male_BL[0] #corr_matrices[0]
matrix2 = male_FU[0] #corr_matrices_fu1[0]

# Function to create block-diagonal matrix from submatrices
def create_block_diagonal_matrix(submatrices, lobe_mapping):
    # Calculate the total number of ROIs
    num_rois = sum(len(roi_list) for roi_list in lobe_mapping.values())
    
    # Initialize the block-diagonal matrix
    block_diagonal_matrix = np.zeros((num_rois, num_rois))
    
    # Fill the block-diagonal matrix with the submatrices
    row_idx = 0
    for lobe, roi_list in lobe_mapping.items():
        size = len(roi_list)
        block_diagonal_matrix[row_idx:row_idx+size, row_idx:row_idx+size] = submatrices[lobe]
        row_idx += size
    
    return block_diagonal_matrix

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Compute the submatrices for each lobe
submatrices = {}
for lobe, roi_list in lobe_mapping.items():
    size = len(roi_list)
    submatrix = np.zeros((size, size))
    for i, roi1 in enumerate(roi_list):
        for j, roi2 in enumerate(roi_list):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]
    submatrices[lobe] = submatrix

# Create the block-diagonal matrix
block_diagonal_matrix = create_block_diagonal_matrix(submatrices, lobe_mapping)

# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]


# Plot heatmap for the block-diagonal matrix
title = "K-operator as a block-diagonal matrix"
plotting.plot_matrix(block_diagonal_matrix, labels=reordered_labels, colorbar=True, vmax=1, vmin=-1, title=title)


print("Size of the block-diagonal matrix:", block_diagonal_matrix.shape)

In [ ]:
# Single blocks of the K operator


# Blocks: submatrices corresponding to the lobes

import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}


matrix1 = male_BL[0] #corr_matrices[0]
matrix2 = male_FU[0] #corr_matrices_fu1[0]

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Function to visualize submatrix for a given lobe
def visualize_submatrix_for_lobe(lobe):
    lobe_indices = lobe_mapping[lobe]
    submatrix = np.zeros((len(lobe_indices), len(lobe_indices)))
    submatrix_labels = [atlas_labels[i] for i in lobe_indices]  # Get ROI labels

    
    for i, roi1 in enumerate(lobe_indices):
        for j, roi2 in enumerate(lobe_indices):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]
    
    # Plot heatmap for the submatrix
    title = f"Heatmap for Submatrix of {lobe}"
    plotting.plot_matrix(submatrix, labels=submatrix_labels, colorbar=True, vmax=1, vmin=-1, title=title)
    plotting.show()

# Visualize submatrices for each lobe
for lobe in lobe_mapping:
    visualize_submatrix_for_lobe(lobe)


